In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! pip install -q kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
#! kaggle datasets download -d tourist55/alzheimers-dataset-4-class-of-images
#!unzip '/content/alzheimers-dataset-4-class-of-images.zip' -d 'kaggle'

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [4]:
! pip install wandb
! wandb login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.3/277.3 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.9 MB/s eta 0:00:00
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
#!wandb login --relogin

In [6]:
%load_ext autoreload
%autoreload 2

import wandb
#from wandb.keras import WandbMetricsLogger
import h5py
import os
import pandas as pd
import numpy as np
from glob import glob
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from pylab import rcParams
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler, PowerTransformer, QuantileTransformer, Normalizer
import tensorflow as tf
from tensorflow import keras
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, AveragePooling2D, BatchNormalization, Input, concatenate
from keras.models import Model, Sequential
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, confusion_matrix
from tensorflow.python.client import device_lib

In [7]:
# Set up constants
DO_LOAD = True
DO_PREPROCESS = True
DO_BALANCE = True
DO_REMOVE_SMALLEST_CATEGORY = False
DO_MERGE_SMALLEST_CATEGORIES = False
DO_PREPROCESSDATA = True
DO_RGB = True

BATCHSIZE = 64
N_EPOCHS = 175
N_CLASSES = 4
MAX_OVERSAMPLING_SEED = 4

In [8]:
def load_data(path):
    folders = os.listdir(path)
    data = []

    for folder in folders:
        files = os.listdir(os.path.join(path, folder))

        for file in files:
            file_path = os.path.join(path, folder, file)
            image = np.asarray(cv2.imread(file_path))
            data.append([image, folder])

    df = pd.DataFrame(data, columns=['image', 'label'])
    print(f"Found {df.shape[0]} images belonging to {len(df['label'].unique())} classes")

    return df

# Preprocess images
def preprocess_images(images, target_size=(88, 104)):
    processed_images = []
    for img in images:
        img = cv2.resize(img, (target_size[1], target_size[0]))  # Resize to (104, 88)
        processed_images.append(img)
    return np.array(processed_images)

# Remove smallest category
def removeSmallestCategory(dataset, dataset_name):
    value_counts = dataset['label'].value_counts()
    min_count_category = value_counts.idxmin()  # Get the label of the category with the smallest count
    newDataset = dataset[dataset['label'] != min_count_category]
    print("Category removed: "+ min_count_category +" from ", dataset_name)
    return newDataset, min_count_category

def removeSmallestGivenCategory(dataset, dataset_name, category):
    value_counts = dataset['label'].value_counts()
    newDataset = dataset[dataset['label'] != category]
    print("Category removed: "+ category +" from ", dataset_name)
    return newDataset

def mergeSmallestCategories(dataset):
    value_counts = dataset['label'].value_counts()
    min_count_categories = value_counts.nsmallest(2).index
    new_label = min_count_categories[0] + '+' + min_count_categories[1]

    dataset['label'] = dataset['label'].replace({min_count_categories[0]: new_label, min_count_categories[1]: new_label})

    print(f"Categories merged: {min_count_categories[0]} and {min_count_categories[1]} into {new_label}")

    return dataset, min_count_categories[0], min_count_categories[1]

def mergeSmallestGivenCategories(dataset, catagory1, catagory2):
    value_counts = dataset['label'].value_counts()
    min_count_categories = [catagory1, catagory2]
    new_label = min_count_categories[0] + '+' + min_count_categories[1]

    dataset['label'] = dataset['label'].replace({min_count_categories[0]: new_label, min_count_categories[1]: new_label})

    print(f"Categories merged: {min_count_categories[0]} and {min_count_categories[1]} into {new_label}")

    return dataset

# Split into test and train
def split_train_test(dataset):
    dataset_train = pd.DataFrame()
    dataset_test = pd.DataFrame()

    for label, group in dataset.groupby('label'):
        train_size = int(0.8 * len(group))
        train_data = group[:train_size]
        test_data = group[train_size:]

        dataset_train = pd.concat([dataset_train, train_data])
        dataset_test = pd.concat([dataset_test, test_data])

    return dataset_train, dataset_test

# Balance the given dataset with oversampling
def balanceDatasetWOverSamp(dataset):
    value_counts = dataset['label'].value_counts()
    max_count = value_counts.max()
    balanced_data = pd.DataFrame()
    for label, group in dataset.groupby('label'):
        if len(group) < max_count:
            sampling_effect = min(MAX_OVERSAMPLING_SEED * len(group), max_count) / len(group)
            print(f"Category {label}: Sampling effect - {sampling_effect}")
            oversampled_group = group.sample(min(MAX_OVERSAMPLING_SEED * len(group), max_count), replace=True, random_state=42)
            balanced_data = pd.concat([balanced_data, oversampled_group])
        else:
            balanced_data = pd.concat([balanced_data, group.sample(max_count)])

    return balanced_data

# Balance the given dataset with undersampling
def balanceDataset(dataset):
    value_counts = dataset['label'].value_counts()
    min_count = value_counts.min()
    balanced_data = pd.DataFrame()

    for label, group in dataset.groupby('label'):
        undersampled_group = group.sample(min_count, random_state=42)
        balanced_data = pd.concat([balanced_data, undersampled_group])

    return balanced_data

def calculateFalsePositive(conf_matrix, class_names):
    # Calculate the false positive rate for each class
    # False Positive Rate = FP / (FP + TN)
    false_positive_rate = []
    for i in range(len(conf_matrix)):
        fp = sum(conf_matrix[:, i]) - conf_matrix[i, i]
        tn = np.sum(conf_matrix) - np.sum(conf_matrix[i, :]) - np.sum(conf_matrix[:, i]) + conf_matrix[i, i]
        fpr = fp / (fp + tn)
        false_positive_rate.append((class_names[i], fpr))
    return false_positive_rate

def calculateFalseNegative(conf_matrix, class_names):
    # Calculate the false negative rate for each class
    # False Negative Rate = FN / (FN + TP)
    false_negative_rate = []
    for i in range(len(conf_matrix)):
        fn = sum(conf_matrix[i, :]) - conf_matrix[i, i]
        tp = conf_matrix[i, i]
        fnr = fn / (fn + tp)
        false_negative_rate.append((class_names[i], fnr))
    return false_negative_rate

In [9]:
# Load the data
data_dir = "drive/MyDrive/Uni Shit/SEP/Dataset/Alzheimer_s Dataset/"

if DO_LOAD:
    print("Loading data...")
    if DO_RGB:
        TRAIN = load_data(os.path.join(data_dir, 'train'))
        TEST = load_data(os.path.join(data_dir, 'test'))
    else:
        TRAIN = load_data(os.path.join(data_dir, 'train_gray'))
        TEST = load_data(os.path.join(data_dir, 'test_gray'))
    print("Data imported.")

Loading data...
Found 5121 images belonging to 4 classes
Found 1279 images belonging to 4 classes
Data imported.


In [10]:
# Initialize a W&B run
wandb.init(
  # Set the project where this run will be logged
  project="GroupProject - Alzheimer's CNN",
  # We pass a run name (otherwise it will be randomly assigned, like sunshine-lollypop-10)
  name=f"experiment_",
  # Track hyperparameters and run metadata
  config={
  #"learning_rate": 0.02,
  "architecture": "CNN",
  "dataset": "Alzheimer's (4 classes)",
  "epochs": N_EPOCHS,
  })

# Original sizes of TRAIN and TEST datasets
train_size = len(TRAIN)
test_size = len(TEST)
print(f"Size of TRAIN dataset: {train_size}")
print(f"Size of TEST dataset: {test_size}")

if DO_REMOVE_SMALLEST_CATEGORY:
    print("Removing smallest category...")
    TRAIN, smallestCat = removeSmallestCategory(TRAIN, "TRAIN")
    TEST = removeSmallestGivenCategory(TEST, "TEST", smallestCat)
    N_CLASSES = N_CLASSES - 1
    print("Done!")

if DO_MERGE_SMALLEST_CATEGORIES:
    print("Merging smallest categories...")
    TRAIN, cat1, cat2 = mergeSmallestCategories(TRAIN)
    TEST = mergeSmallestGivenCategories(TEST, cat1, cat2)
    N_CLASSES = N_CLASSES - 1
    print("Done!")

# Print size of each category in TRAIN dataset before rebalancing
print("Category sizes in TRAIN dataset before rebalancing:")
print(TRAIN['label'].value_counts())

# Balance and split the TRAIN dataset
train_data_train, train_data_test = split_train_test(TRAIN)

if DO_BALANCE:
    train_data_train = balanceDatasetWOverSamp(train_data_train)
    train_data_test = balanceDataset(train_data_test)
else:
    print("Skipping balancing for TRAIN dataset...")

# Print size of each category in TRAIN dataset after rebalancing
print("Category sizes in TRAIN dataset after rebalancing (excluding testing portion):")
print(train_data_train['label'].value_counts())
print("Category sizes of test portion of TRAIN dataset:")
print(train_data_test['label'].value_counts())

# Print size of each category in TEST dataset before rebalancing
print("Category sizes in TEST dataset before rebalancing:")
print(TEST['label'].value_counts())

# Balance and split the TEST dataset
test_data_train, test_data_test = split_train_test(TEST)

if DO_BALANCE:
    test_data_train = balanceDatasetWOverSamp(test_data_train)
    test_data_test = balanceDataset(test_data_test)
else:
    print("Skipping balancing for TEST dataset...")

# Print size of each category in TEST dataset after rebalancing
print("Category sizes in TEST dataset after rebalancing (excluding testing portion):")
print(test_data_train['label'].value_counts())
print("Catagory sizes of test portion of TEST dataset:")
print(test_data_test['label'].value_counts())

# Combine the balanced TRAIN and TEST datasets
print("Combining balanced TRAIN and TEST datasets...")
combined_train = pd.concat([train_data_train, test_data_train])
combined_test = pd.concat([train_data_test, test_data_test])

print("Final combined TRAIN dataset:")
print(combined_train['label'].value_counts())
print("Final combined TEST dataset:")
print(combined_test['label'].value_counts())

# Shuffle the combined datasets
print("Shuffling combined TRAIN and TEST datasets...")
combined_train = combined_train.sample(frac=1).reset_index(drop=True)
combined_test = combined_test.sample(frac=1).reset_index(drop=True)

# Separate images and labels
TRAIN_processed = preprocess_images(combined_train['image'].values)
TEST_processed = preprocess_images(combined_test['image'].values)

# One-hot encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(combined_train['label'])
train_labels_categorical = to_categorical(train_labels_encoded, num_classes=N_CLASSES)
test_labels_encoded = label_encoder.transform(combined_test['label'])
test_labels_categorical = to_categorical(test_labels_encoded, num_classes=N_CLASSES)
train_labels_categorical_tensor = tf.constant(train_labels_categorical)
test_labels_categorical_tensor = tf.constant(test_labels_categorical)

TRAIN_tensor = tf.convert_to_tensor(TRAIN_processed)
TEST_tensor = tf.convert_to_tensor(TEST_processed)

# Definition of blocks of layers
def preprocessingBlock(rotation=0.2, zoom=0.2, contrast=0.2):
    block = tf.keras.Sequential()
    block.add(tf.keras.layers.Rescaling(1./255))
    #block.add(tf.keras.layers.RandomFlip("horizontal"))
    block.add(tf.keras.layers.RandomContrast(contrast))
    block.add(tf.keras.layers.RandomZoom(zoom))
    block.add(tf.keras.layers.RandomRotation(rotation))
    return block

def denseBlock(units, dropout_rate):
    block = tf.keras.Sequential([
        tf.keras.layers.Dense(units, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout_rate)
    ])

    return block


wandb: Currently logged in as: ehstone2308 (elliott-stone). Use `wandb login --relogin` to force relogin


Size of TRAIN dataset: 5121
Size of TEST dataset: 1279
Category sizes in TRAIN dataset before rebalancing:
label
NonDemented         2560
VeryMildDemented    1792
MildDemented         717
ModerateDemented      52
Name: count, dtype: int64
Category MildDemented: Sampling effect - 3.574171029668412
Category ModerateDemented: Sampling effect - 4.0
Category VeryMildDemented: Sampling effect - 1.4291695743196091
Category sizes in TRAIN dataset after rebalancing (excluding testing portion):
label
MildDemented        2048
NonDemented         2048
VeryMildDemented    2048
ModerateDemented     164
Name: count, dtype: int64
Category sizes of test portion of TRAIN dataset:
label
MildDemented        11
ModerateDemented    11
NonDemented         11
VeryMildDemented    11
Name: count, dtype: int64
Category sizes in TEST dataset before rebalancing:
label
NonDemented         640
VeryMildDemented    448
MildDemented        179
ModerateDemented     12
Name: count, dtype: int64
Category MildDemented: Sam

In [15]:
# Define the model architecture
input_layer = keras.layers.Input(shape=(88, 104, 3)) #88 x 104 px with three colour layers (RGB) for JPEG
aug = preprocessingBlock(rotation=0.1, zoom=0.1, contrast=0.1)(input_layer)
conv1 = keras.layers.Conv2D(32, (3, 3), activation='relu')(aug)
pool1 = keras.layers.MaxPooling2D((2, 2))(conv1)
conv2 = keras.layers.Conv2D(64, (3, 3), activation='relu')(pool1)
pool2 = keras.layers.MaxPooling2D((2, 2))(conv2)
conv3 = keras.layers.Conv2D(128, (3, 3), activation='relu')(pool2)
pool3 = keras.layers.MaxPooling2D((2, 2))(conv3)
flatten = keras.layers.Flatten()(pool3)
dense1 = keras.layers.Dense(128, activation='relu')(flatten)
output_layer = Dense(N_CLASSES, activation='softmax')(dense1)
model = keras.models.Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='categorical_crossentropy',
  metrics=[
    keras.metrics.CategoricalAccuracy(),
    keras.metrics.F1Score(average='weighted'),
    keras.metrics.FalseNegatives(),
    keras.metrics.FalsePositives(),
    #keras.metrics.IoU(),
    ],)

# Train the model on the training data and validate on the testing data
model.fit(TRAIN_tensor, train_labels_categorical_tensor, validation_data=(TEST_tensor, test_labels_categorical_tensor), epochs=25)#, callbacks=metrics)#[WandbMetricsLogger()])

Epoch 1/25
247/247 [==============================] - 7s 14ms/step - loss: 1.1419 - categorical_accuracy: 0.4289 - f1_score: 0.4152 - false_negatives_4: 6868.0000 - false_positives_4: 742.0000 - val_loss: 1.7914 - val_categorical_accuracy: 0.3214 - val_f1_score: 0.2382 - val_false_negatives_4: 50.0000 - val_false_positives_4: 16.0000
Epoch 2/25
247/247 [==============================] - 3s 13ms/step - loss: 1.0175 - categorical_accuracy: 0.5147 - f1_score: 0.4905 - false_negatives_4: 5587.0000 - false_positives_4: 1376.0000 - val_loss: 1.7889 - val_categorical_accuracy: 0.2500 - val_f1_score: 0.2154 - val_false_negatives_4: 50.0000 - val_false_positives_4: 15.0000
Epoch 3/25
247/247 [==============================] - 3s 13ms/step - loss: 0.9598 - categorical_accuracy: 0.5499 - f1_score: 0.5313 - false_negatives_4: 5174.0000 - false_positives_4: 1419.0000 - val_loss: 1.8702 - val_categorical_accuracy: 0.2857 - val_f1_score: 0.2176 - val_false_negatives_4: 47.0000 - val_false_positives_4

In [16]:
# Pretrained model - Xception
inputs = keras.layers.Input(shape=(88, 104, 3))
aug = preprocessingBlock(rotation=0.1, zoom=0.1, contrast=0.1)(inputs)
xception = keras.applications.Xception(include_top=False, weights='imagenet')(aug)
globalavg = keras.layers.GlobalAveragePooling2D()(xception)
denseblock1 = denseBlock(1024, 0.3)(globalavg)
denseblock2 = denseBlock(512, 0.2)(denseblock1)
denseblock3 = denseBlock(128, 0.2)(denseblock2)
outputs = keras.layers.Dense(N_CLASSES, activation='softmax')(denseblock3)
pretrained = keras.Model(inputs, outputs)
pretrained.compile(optimizer='adam', loss='categorical_crossentropy',
  metrics=[
    keras.metrics.CategoricalAccuracy(),
    keras.metrics.F1Score(average = 'weighted'),
    keras.metrics.FalseNegatives(),
    keras.metrics.FalsePositives(),
    #keras.metrics.IoU(),
    ],)
pretrained.fit(TRAIN_tensor, train_labels_categorical_tensor, validation_data=(TEST_tensor, test_labels_categorical_tensor), epochs=25)#, callbacks=metrics)#[WandbMetricsLogger()])

Epoch 1/25
247/247 [==============================] - 55s 119ms/step - loss: 1.2731 - categorical_accuracy: 0.4451 - f1_score: 0.4360 - false_negatives_5: 5795.0000 - false_positives_5: 2017.0000 - val_loss: 2.2140 - val_categorical_accuracy: 0.2857 - val_f1_score: 0.1718 - val_false_negatives_5: 41.0000 - val_false_positives_5: 34.0000
Epoch 2/25
247/247 [==============================] - 28s 114ms/step - loss: 1.0491 - categorical_accuracy: 0.5254 - f1_score: 0.5171 - false_negatives_5: 5301.0000 - false_positives_5: 1528.0000 - val_loss: 2.0397 - val_categorical_accuracy: 0.2679 - val_f1_score: 0.2012 - val_false_negatives_5: 43.0000 - val_false_positives_5: 17.0000
Epoch 3/25
247/247 [==============================] - 28s 115ms/step - loss: 0.9101 - categorical_accuracy: 0.6071 - f1_score: 0.6017 - false_negatives_5: 4263.0000 - false_positives_5: 1693.0000 - val_loss: 2.4338 - val_categorical_accuracy: 0.2679 - val_f1_score: 0.1342 - val_false_negatives_5: 42.0000 - val_false_posi

In [20]:
# Close the W&B run (Might not save the final scores of the model, as only taking the last training values, not the final scores of the model)
wandb.finish()

# Evaluate the model on the testing data
test_loss = model.evaluate(TEST_tensor, test_labels_categorical_tensor, batch_size=BATCHSIZE)
print(f'Test loss: {test_loss}')#, Test accuracy: {test_accuracy}')

test_loss_p = pretrained.evaluate(TEST_tensor, test_labels_categorical_tensor, batch_size=BATCHSIZE)
print(f'Xception Test loss: {test_loss}')#, Xception Test accuracy: {test_accuracy}')

# Make predictions on the test data
predictions = model.predict(TEST_tensor)
predicted_labels = np.argmax(predictions, axis=1)
# Convert one-hot encoded test labels back to categorical labels
test_labels = np.argmax(test_labels_categorical, axis=1)
# Calculate the F1 score
f1score = f1_score(test_labels, predicted_labels, average='weighted')
# Calculate the confusion matrix
conf_matrix = confusion_matrix(test_labels, predicted_labels)

print('\nXception:')
predictions_p = pretrained.predict(TEST_tensor)
predicted_labels_p = np.argmax(predictions_p, axis=1)
f1score_p = f1_score(test_labels, predicted_labels_p, average='weighted')
conf_matrix_p = confusion_matrix(test_labels, predicted_labels)

# After calculating the confusion matrix
class_names = combined_train['label'].unique().tolist()
#false_positive_rate = calculateFalsePositive(conf_matrix, class_names)
#false_negative_rate = calculateFalseNegative(conf_matrix, class_names)
#print(f'False Positive Rate for each class: {false_positive_rate}')
#print(f'False Negative Rate for each class: {false_negative_rate}')

print(f'F1 Score: {f1score}')

# After calculating the confusion matrix
print('\nXception:')
#false_positive_rate_p = calculateFalsePositive(conf_matrix_p, class_names)
#false_negative_rate_p = calculateFalseNegative(conf_matrix_p, class_names)
#print(f'False Positive Rate for each class: {false_positive_rate_p}')
#print(f'False Negative Rate for each class: {false_negative_rate_p}')

print(f'F1 Score: {f1score_p}')

# Save the trained model
model.save('drive/MyDrive/Uni Shit/SEP/my_model.keras')
pretrained.save('drive/MyDrive/Uni Shit/SEP/xception_model.keras')

1/1 [==============================] - 0s 43ms/step - loss: 0.6501 - categorical_accuracy: 0.8393 - f1_score: 0.8378 - false_negatives_4: 9.0000 - false_positives_4: 9.0000
Test loss: [0.6500678062438965, 0.8392857313156128, 0.837780773639679, 9.0, 9.0]
1/1 [==============================] - 0s 104ms/step - loss: 0.8568 - categorical_accuracy: 0.7500 - f1_score: 0.7043 - false_negatives_5: 14.0000 - false_positives_5: 14.0000
Xception Test loss: [0.6500678062438965, 0.8392857313156128, 0.837780773639679, 9.0, 9.0]
2/2 [==============================] - 0s 5ms/step

Xception:
2/2 [==============================] - 0s 43ms/step
F1 Score: 0.8377807974582169

Xception:
F1 Score: 0.7043190249072602
